In [ ]:
!pip install datasets==2.16.0
!pip install huggingface-hub==0.20.0
!apt-get install -y libsox-dev
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16
  Attempting uninstall: datasets
    Fou

In [ ]:
# Keyword‑Spotting project

from __future__ import annotations
import json, os, random
from pathlib import Path
from typing import Tuple, Dict

import torch, torchaudio
import torch.nn as nn
import os
import torch.nn.functional as F
import torchvision.models as tvm  # MobileNet V2 backbone
import torchaudio.transforms as T
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from tqdm.notebook import tqdm

# -------- Save parameters to Drive --------
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class WaveToSpec:
    """Waveform ➜ log‑Mel **or** MFCC tensor.

    Parameters
    ----------
    feature_type : "mel" | "mfcc"
        If "mfcc", `n_mfcc` controls the number of cepstral coeffs.
    normalize : bool | tuple(float,float)
        *False* – return raw log power values.
        *True*  – per‑sample standard score.
        *(mean,std)* – dataset‑wide constants.
    """
    def __init__(self,
                 feature_type: str = "mel",
                 sample_rate: int = 16_000,
                 n_fft: int = 2048,
                 hop_length: int = 128,
                 n_mels: int = 128,
                 n_mfcc: int = 40,
                 top_db: int | None = 80,
                 apply_mask: bool = True,
                 freq_mask_param: int = 15,
                 time_mask_param: int = 10):
        self.feature_type = feature_type.lower(); assert self.feature_type in {"mel","mfcc"}
        self.apply_mask = apply_mask and self.feature_type == "mel"

        if self.feature_type == "mel":
            self.spec = T.MelSpectrogram(sample_rate, n_fft, hop_length, n_mels, power=2)
            self.to_db = T.AmplitudeToDB(stype="power", top_db=top_db)
            if self.apply_mask:
                self.freq_mask = T.FrequencyMasking(freq_mask_param)
                self.time_mask = T.TimeMasking(time_mask_param)
        else:
            self.spec = T.MFCC(sample_rate, n_mfcc,
                                melkwargs=dict(n_fft=n_fft, hop_length=hop_length, n_mels=n_mels))
            self.to_db = None
            self.freq_mask = self.time_mask = None

    def __call__(self, wav: torch.Tensor) -> torch.Tensor:
        if wav.dim() == 1:
            wav = wav.unsqueeze(0)
        feats = self.spec(wav)
        if self.apply_mask:
            feats = self.freq_mask(feats); feats = self.time_mask(feats)
            feats = self.freq_mask(feats); feats = self.time_mask(feats)
        if self.to_db is not None:
            feats = self.to_db(feats.clamp(min=1e-10))
        return feats   # [1, F, T]

# -------------------------
# 2) Waveform-level augs
# -------------------------
class Augment:
    def __init__(self, stretch: Tuple[float,float]=(1.0,1.0),
                 shift_ms: int = 100,
                 noise: Tuple[float,float]=(0.,0.005),
                 sr: int = 16_000):
        self.stretch = stretch
        self.shift   = int(shift_ms * sr / 1000)
        self.noise   = noise
        self.sr      = sr

    def _shift(self, x: torch.Tensor):
        if self.shift == 0:
            return x
        s = int(torch.randint(-self.shift, self.shift + 1, ()).item())
        if s == 0:
            return x
        return (F.pad(x, (s, 0))[:, :-s] if s > 0 else F.pad(x, (0, -s))[:, -s:])

    def __call__(self, wav: torch.Tensor):
        squeezed = False
        if wav.dim() == 1:
            wav = wav.unsqueeze(0); squeezed = True

        # Stretch remains OFF because (1.0, 1.0)
        wav = self._shift(wav)

        if self.noise[1] > 0:
            sigma = float(torch.empty(()).uniform_(*self.noise))
            if sigma > 0:
                wav = wav + sigma * torch.randn_like(wav)

        return wav.squeeze(0) if squeezed else wav

In [ ]:
# -------------------------
# 3) Dataset + Collate
# -------------------------
class SpeechCommands(Dataset):
    """HF split → (feature, label) with fixed waveform length (like your Mamba)."""
    def __init__(self, hf_split, aug: Augment | None, frontend: WaveToSpec, wav_len: int = 16_000):  # CHANGED: wav_len
        self.ds, self.aug, self.front = hf_split, aug, frontend
        self.wav_len = wav_len  # pad/crop wave to this length before spec

    def __len__(self): return len(self.ds)

    def __getitem__(self, idx):
        item = self.ds[idx]
        wav = torch.from_numpy(item["audio"]["array"]).float()

        # CHANGED: fixed wav_len like Mamba
        if wav.numel() < self.wav_len:
            wav = F.pad(wav, (0, self.wav_len - wav.numel()))
        else:
            wav = wav[: self.wav_len]

        if self.aug:
            wav = self.aug(wav)

        feats = self.front(wav)                          # [1, F, T]
        feats = (feats - feats.mean()) / (feats.std() + 1e-6)
        return feats, item["label"]

def collate_pad(batch):
    feats, labels = zip(*batch)
    B, F = len(feats), feats[0].size(1)
    T_max = max(x.size(-1) for x in feats)
    out = feats[0].new_zeros(B, 1, F, T_max)
    for i, x in enumerate(feats):
        T = x.size(-1)
        out[i, :, :, :T] = x
    return out, torch.tensor(labels, dtype=torch.long)


# -------------------------
# 4) Model
# -------------------------
class KeywordCNN(nn.Module):
    """Small CNN baseline with time-preserving pool and dropout head."""
    def __init__(self, n_classes:int):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(),
            nn.MaxPool2d((2,1)),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),
        )
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(128, n_classes)
    def forward(self, x):
        x = self.features(x).flatten(1)
        x = self.dropout(x)
        return self.fc(x)

def build_mobilenet_v2(num_classes: int, alpha: float = 0.75, pretrained: bool = False) -> nn.Module:
    import torchvision.models as tvm
    net = tvm.mobilenet_v2(width_mult=alpha,
                           weights=None if not pretrained else tvm.MobileNet_V2_Weights.DEFAULT)
    # adapt first conv to 1 channel
    first = net.features[0][0]
    net.features[0][0] = nn.Conv2d(1, first.out_channels,
                                   kernel_size=first.kernel_size,
                                   stride=first.stride,
                                   padding=first.padding,
                                   bias=False)
    # simple dropout + classifier
    in_feats = net.classifier[-1].in_features
    net.classifier = nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(in_feats, num_classes)
    )
    return net

# -------------------------
# 5) Eval helper
# -------------------------
@torch.no_grad()
def evaluate(model, loader, device, criterion):
    model.eval(); tot=correct=loss_sum=0
    for xb,yb in loader:
        xb,yb = xb.to(device), yb.to(device)
        logits = model(xb)
        loss = criterion(logits,yb)
        loss_sum += loss.item()*xb.size(0)
        correct  += (logits.argmax(1)==yb).sum().item(); tot += xb.size(0)
    return loss_sum/tot, 100.0*correct/tot

In [ ]:
import math
# -------------------------
# 6) Main
# -------------------------
if __name__ == "__main__":
    # ---- dataset
    ds = load_dataset("google/speech_commands", "v0.02")
    labels = ds["train"].features["label"].names
    n_classes = len(labels)

    feature_type = "mel"               # "mel" or "mfcc"
    Epochs = 100
    base_lr = 1e-3
    warmup_frac = 0.1
    wav_len = 16_000

    # train/eval frontends (SpecAugment only on train)
    frontend_train = WaveToSpec(
        feature_type=feature_type,
        n_mfcc=40, n_mels=128,
        apply_mask=True,
        freq_mask_param=14, time_mask_param=24
    )
    frontend_eval = WaveToSpec(
        feature_type=feature_type,
        n_mfcc=40, n_mels=128,
        apply_mask=False
    )

    # augs: shift + small noise; stretch OFF by default (1.0,1.0)
    aug = Augment(stretch=(1.0, 1.0), shift_ms=100, noise=(0., 0.005))

    train_ds = SpeechCommands(ds["train"],      aug,  frontend_train, wav_len=wav_len)
    val_ds   = SpeechCommands(ds["validation"], None, frontend_eval,  wav_len=wav_len)
    test_ds  = SpeechCommands(ds["test"],       None, frontend_eval,  wav_len=wav_len)

    dl_kwargs = dict(batch_size=128, num_workers=4, pin_memory=True, collate_fn=collate_pad)
    train_dl  = DataLoader(train_ds, shuffle=True,  **dl_kwargs)
    val_dl    = DataLoader(val_ds,   shuffle=False, **dl_kwargs)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    use_amp = (device.type == "cuda")
    scaler = torch.amp.GradScaler(device.type, enabled=use_amp)

    model = build_mobilenet_v2(n_classes, alpha=0.75, pretrained=False).to(device)

    criterion = nn.CrossEntropyLoss(label_smoothing=0.05)
    opt = torch.optim.AdamW(model.parameters(), lr=base_lr, weight_decay=5e-5, betas=(0.9, 0.999))

    # warmup + cosine tail + Plateau (after warmup)
    steps_per_epoch = len(train_dl)
    true_steps      = steps_per_epoch * Epochs
    total_steps     = steps_per_epoch * int(Epochs * 1.5)
    warmup_steps    = int(true_steps * warmup_frac)

    def lr_lambda(step):
        if step < warmup_steps:
            return float(step) / float(max(1, warmup_steps))
        progress = float(step - warmup_steps) / float(max(1, total_steps - warmup_steps))
        return max(0.01, 0.5 * (1.0 + math.cos(math.pi * progress)))

    sched = torch.optim.lr_scheduler.LambdaLR(opt, lr_lambda)
    plateau = torch.optim.lr_scheduler.ReduceLROnPlateau(
        opt, mode="min", factor=0.5, patience=3, threshold=1e-3, cooldown=0, min_lr=1e-6
    )
    prev_plateau_lr = opt.param_groups[0]['lr']

    best_val_acc = 0.0
    BEST_PATH = Path("/content/best_kws_cnn.pt")
    CKPT_DIR = Path("/content/drive/MyDrive/kws_models")
    CKPT_DIR.mkdir(parents=True, exist_ok=True)

    global_step = 0
    for epoch in range(1, Epochs + 1):
        model.train()
        running_loss = correct = total = 0
        pbar = tqdm(train_dl, desc=f"Epoch {epoch:02d}")

        for xb, yb in pbar:
            xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
            with torch.amp.autocast(device.type, enabled=use_amp):
                logits = model(xb)
                loss = criterion(logits, yb)
                if not torch.isfinite(loss):
                    continue

            opt.zero_grad(set_to_none=True)
            scaler.scale(loss).backward()
            scaler.unscale_(opt)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.3)
            scaler.step(opt); scaler.update()
            sched.step()   # per-batch
            global_step += 1

            pred = logits.argmax(1)
            correct += (pred == yb).sum().item()
            total += yb.size(0)
            running_loss += loss.item() * yb.size(0)
            pbar.set_postfix(
                train_loss=f"{running_loss/max(1,total):.3f}",
                train_acc=f"{100*correct/max(1,total):.1f}%",
                lr=f"{opt.param_groups[0]['lr']:.2e}"
            )

        val_loss, val_acc = evaluate(model, val_dl, device, criterion)
        print(f"Epoch {epoch:02d} ➜ train {100*correct/max(1,total):.1f}% | "
              f"val {val_acc:.1f}% (loss {val_loss:.3f}) | lr {opt.param_groups[0]['lr']:.2e}")

        # After warmup, allow Plateau to adjust LR (epoch-level)
        if global_step >= warmup_steps:
            plateau.step(val_loss)
            new_lr = opt.param_groups[0]['lr']
            if new_lr < prev_plateau_lr - 1e-12:
                print(f"[Plateau] LR reduced: {prev_plateau_lr:.2e} -> {new_lr:.2e}")
            prev_plateau_lr = new_lr

        # Save best-by-accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), BEST_PATH)
            print(f"Saved new BEST CNN @ {best_val_acc:.1f}%")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/_login.py:337: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/datasets/load.py:1429: FutureWarning: The repository for google/speech_commands contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/speech_commands
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
 

Epoch 01:   0%|          | 0/663 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch 01 ➜ train 4.0% | val 6.0% (loss 3.478) | lr 1.00e-04
★ Saved new BEST CNN @ 6.0%


Epoch 02:   0%|          | 0/663 [00:00<?, ?it/s]

...

Epoch 98 ➜ train 96.5% | val 96.4% (loss 0.491) | lr 3.03e-04
[Plateau] LR reduced: 3.14e-04 -> 3.03e-04


Epoch 99:   0%|          | 0/663 [00:00<?, ?it/s]

Epoch 99 ➜ train 96.4% | val 96.4% (loss 0.489) | lr 2.93e-04
[Plateau] LR reduced: 3.03e-04 -> 2.93e-04


Epoch 100:   0%|          | 0/663 [00:00<?, ?it/s]

Epoch 100 ➜ train 96.6% | val 96.4% (loss 0.491) | lr 2.83e-04
[Plateau] LR reduced: 2.93e-04 -> 1.42e-04


In [ ]:
    # ---- save LAST weights to Drive ----
    torch.save(model.state_dict(), "/content/last_cnn_raw.pt")
    torch.save(model.state_dict(), CKPT_DIR / "last_cnn_raw.pt")
    if BEST_PATH.exists():
        import shutil
        shutil.copy2(BEST_PATH, CKPT_DIR / "best_cnn.pt")
        print(f"Copied BEST CNN to Drive: {CKPT_DIR/'best_cnn.pt'}")

Copied BEST CNN to Drive: /content/drive/MyDrive/kws_models/best_cnn.pt
